[toc]

# 安装

```sh
pip install --upgrade pyinstaller
```

安装最新开发版

```sh
pip install https://github.com/pyinstaller/pyinstaller/tarball/develop
```

验证安装

```sh
pyinstaller -v
```

结果应当输出类似与 3.x 或者 3.n.dev0-xxxxxx 的开发版本号

## 💊温馨提示

1. **在虚拟环境下打包**，只安装程序需要的依赖包，避免打包后文件过大
2. 在打包成单个 exe 文件之前，请先尝试打包成**一个文件夹**测试是否正常运行
3. 打包成单个文件启动较慢，原因是程序需要按原目录结构解压到临时目录下，在用户目录下生成临时文件，结束后会自动删除

# 用法

## 基本用法

### 1. 直接打包

```sh
pyinstaller  myscript.py
```

### 2. 打包成单文件

```sh
pyinstaller -F myscript.py
```

### 3. 去掉控制台窗口，黑窗口

不显示控制台窗口(类似 cmd 的黑框框), 如果你写的是带 UI 的程序, 此选项基本必选.如果入口程序是 pyw 文件, 此选项默认生效.

```sh
pyinstaller -w myscript.py
```

### 4. 添加图标

```sh
pyinstaller -i icon.ico myscript.py
```

## 高级用法

打包一遍后，会生成一个`xxx.spec`文件, 是一个打包参数配置文件，该文件内容由上一次打包命令决定，可以通过添加打包参数重新生成，也可以可以直接修改其内容供下次使用。

😘 注： spec 文件生成方法

    将 `pyinstaller` 命令换成 `pyi-makespec`

    例：
    ```sh
    pyi-makespec -F filename
    ```

💊 注：当你修改过 spec 文件并且想将其应用，你需要以下命令

    ```sh
    pyinstaller ***.spec
    ```

### 1. 隐式导入

有些模块在代码里是动态加载的，比如通过 `__import__(xxx)` 或者 `importlib` 模块加载的，在打包过程中是无法检测出来的，但是不打包进去这些模块，程序会抛出 `ModuleNotFoundError: No module named 'xxx'` 异常，这时候你有两种方法解决。一是，在代码里显式地 import 这些模块，尽管你不去使用（pycharm 里显示灰色），二是在打包时添加打包参数，可以通过命令行参数，也可通过改 xxx.spec 配置文件。

#### 命令行模式：

可以多次使用

> --hidden-import MODULENAME1, --hiddenimport MODULENAME2

例：

```sh
--hidden-import docx --hidden-import Pillow
```

#### 修改 spec 模式：

> hiddenimports=['docx'],

例：

```ini
hiddenimports=['docx', 'Pillow']
```

### 2. 打包静态文件

当你程序中需要读取一些静态文件比如 `setting.json` ,就需要你将他们一起打包进去，否则 pyinstaller 也会忽略他们。

#### 命令行模式

可以多次使用

```sh
--add-data <SRC;DEST or SRC:DEST>
```

👉 注意：格式为 一个原文件名和目**标文件夹**名！，中间用一个分号或者冒号分割。

👉 注意：路径中需要用 双反斜杠！！

例：  
将原路径（绝对路径或者相对路径都可）setting 目录下的 aaa.json 文件打包到目标的 setting 文件夹下

```sh
--add-data ".\\setting\\aaa.json;.\\setting"
```

例：  
将原路径（绝对路径或者相对路径都可）config 目录下的所有文件 文件打包到目标的 config 文件夹下

```sh
--add-data ".\\config\\*;.\\config"
```

#### 修改 spec 模式

```py
datas=[('.\\config\\*', '.\\config'),
('.\\setting\\aaa.json', '\\setting') ],
```

#### 💊 注意事项

将文件打包成单文件时（但文件夹除外），你在程序中访问这些静态文件时**切勿使用相对路径**，因为单文件在运行时先解压所有需要的文件到 用户临时文件夹下（ /temp 目录），并不在你当前运行路径下寻找这些静态文件。你需要将程序里访问这些静态文件的路径改为绝对路径，才能在其解压出的临时路径下找到他们：

```py
if getattr(sys, 'frozen', None):
    basedir = sys._MEIPASS
else:
    basedir = os.path.dirname(__file__)

#接上例，打包进去的 aaa.json，加到了这个绝对目录。
aaa = os.path.join(basedir, 'setting', 'aaa.json')
```

👉 效果参考：

https://blog.csdn.net/Iv_zzy/article/details/107407167

## 3. 打包二进制依赖文件

将程序中依赖的 dll 或者 so 文件 一起打包进去。（编译好的 python 可调用模块或者 ctypes 加载的 dll 文件都可）

> --add-binary <SRC;DEST or SRC:DEST>

可以多次使用；参数格式参考上一小标题（打包静态文件）

例：

```sh
--add-binary D:\\test\\pack\\HCNetSDK.dll;.\\lib
```

### 4. 加密字节码

详细介绍：https://pyinstaller.readthedocs.io/en/stable/usage.html#encrypting-python-bytecode

使用 --key 参数 指定一个长度为 16 的字符串，来加密 python 字节码文件，
你需要先执行以下命令

```sh
pip install pyinstaller[encryption]
```

然后

```sh
pyinstaller.exe --key=xxxx -F hellow.py
```

（🙏 过程可能不太顺利，过程慢长，打包完文件也较大）
